In [1]:
import keras
import tqdm
import os

import tensorflow as tf
import pandas as pd
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Conv2D
from keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split

from PIL import Image

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
image_name = []

dir = os.listdir('/kaggle/input/arknigth/data')

In [4]:
for i in tqdm.tqdm(dir):
  for dirname, _, filenames in os.walk(f'/kaggle/input/arknigth/data/{i}'):
    for filename in filenames:
      image = os.path.join(dirname, filename)

      image_name.append(image)

print('GOAL')

100%|██████████| 10/10 [00:00<00:00, 13.57it/s]

GOAL


In [5]:
dir

['end',
 'black',
 'sanity',
 'team',
 'battle',
 'end_icon',
 'battle_load_gray',
 'battle_end',
 'main',
 'battle_load_color']

In [6]:
index_arr = []

def text_to_index(path):
    index = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    for i in range(len(dir)):
        if dir[i] in path:
            index[i] = 1
            break

    return np.array(index)

for i in image_name:
  index_arr.append(text_to_index(i))

In [7]:
len(image_name)

4795

In [8]:
X_train, X_test, y_train, y_test = train_test_split(image_name, index_arr, test_size=.2)

In [9]:
dataset_all = tf.data.Dataset.from_tensor_slices((image_name, index_arr))

In [10]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))

dataset_test = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [11]:
next(iter(dataset_train))

(<tf.Tensor: shape=(), dtype=string, numpy=b'/kaggle/input/arknigth/data/team/image_18_48_00_312226.png'>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0])>)

In [12]:
BUFFER_SIZE = 10000
BATCH_SIZE = 32
IMG_WIDTH = 64
IMG_HEIGHT = 64

def norm(img):
    img = tf.cast(img, tf.float32)

    img = (img / 127.5) - 1

    return img

def resize_image(img):
    img = tf.image.resize(
        img, (IMG_WIDTH, IMG_HEIGHT), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR
    )

    return img

def load(image_file):
  image = tf.io.read_file(image_file)
  image = tf.io.decode_jpeg(image)

  input_image = tf.cast(image, tf.float32)

  return input_image

def preprocess_image_train(image, y):
    image = resize_image(load(image))

    image = norm(image)

    return image, y


dataset_norm = lambda x: x.map(preprocess_image_train, num_parallel_calls=tf.data.AUTOTUNE)\
    .shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

dataset_norm_2 = lambda x: x.map(preprocess_image_train, num_parallel_calls=tf.data.AUTOTUNE)\
    .shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


train_dataset = dataset_norm(dataset_train)
test_dataset = dataset_norm_2(dataset_test)

In [13]:
datasets_alls =dataset_norm(dataset_all)

In [14]:
class Residual(keras.layers.Layer):
  def __init__(self, filter_size, padding, strides, conv2d_plus=False, **kwargs):
    super(Residual, self).__init__(**kwargs)

    self.filter_size = filter_size
    self.padding = padding
    self.strides = strides
    self.conv2d_plus = conv2d_plus

    self.conv2d = Conv2D(filter_size ,kernel_size=1, padding=padding, strides=strides)
    self.conv2d_2 = Conv2D(filter_size ,kernel_size=1, padding=padding)
    self.batch_norm = BatchNormalization()
    self.batch_norm_2 = BatchNormalization()

    self.conv2d_3 = None

    if conv2d_plus:
      self.conv2d_3 = Conv2D(filter_size ,kernel_size=3, padding=padding, strides=strides)

    self.activation = keras.layers.ReLU()
    self.activation_2 = keras.layers.ReLU()

  def call(self, x):
    conv = self.activation(self.batch_norm(self.conv2d(x)))
    conv_2 = self.conv2d_2(conv)

    if self.conv2d_3 is not None:
      x = self.conv2d_3(x)

    y = self.batch_norm_2(conv_2)

    y += x

    return self.activation_2(y)

  def get_config(self):
    config = super(Residual, self).get_config()

    config['filter_size'] = self.filter_size
    config['padding'] = self.padding
    config['strides'] = self.strides
    config['conv2d_plus'] = self.conv2d_plus

    return config

In [15]:
r = Residual(6, 'same', 2, True)
test = tf.random.normal((4, 6, 6, 3))
r(test).shape

TensorShape([4, 3, 3, 6])

In [17]:
filter = 64
padding = 'same'
strides = 1
kernel_size = 3
dropout = .1

model = keras.Sequential([
    keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, 3)),
    keras.layers.Conv2D(filter, padding=padding, strides=strides+1, kernel_size=7),
    keras.layers.MaxPooling2D(3),
    Residual(filter, padding, strides),
    Residual(filter, padding, strides),

    keras.layers.Dropout(dropout),

    Residual(filter, padding, strides+1, True),
    Residual(filter, padding, strides),

    keras.layers.Dropout(dropout),

    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dropout(dropout),
    keras.layers.Dense(len(dir), activation='softmax')
])

In [18]:
x = tf.random.normal((64, 64, 64, 3))

In [19]:
model(x).shape

TensorShape([64, 10])

In [21]:
loss_object = keras.losses.CategoricalCrossentropy()
optimizer = keras.optimizers.Adam()

In [24]:
model.compile(loss=loss_object, optimizer=optimizer, metrics=['acc'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 32, 32, 64)        9472      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 64)       0         
 2D)                                                             
                                                                 
 residual_5 (Residual)       (None, 10, 10, 64)        8832      
                                                                 
 residual_6 (Residual)       (None, 10, 10, 64)        8832      
                                                                 
 dropout_2 (Dropout)         (None, 10, 10, 64)        0         
                                                                 
 residual_7 (Residual)       (None, 5, 5, 64)          45760     
                                                        

In [25]:
model.fit(train_dataset, epochs=5, validation_data=test_dataset)

Epoch 1/5


2023-10-17 16:17:23.631763: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout_2/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


120/120 [==============================] - 424s 712ms/step - loss: 0.2833 - acc: 0.9103 - val_loss: 0.3174 - val_acc: 0.8655
Epoch 2/5
120/120 [==============================] - 415s 717ms/step - loss: 0.0135 - acc: 0.9969 - val_loss: 0.0207 - val_acc: 0.9969
Epoch 3/5
120/120 [==============================] - 420s 707ms/step - loss: 0.0530 - acc: 0.9851 - val_loss: 0.5037 - val_acc: 0.8519
Epoch 4/5
120/120 [==============================] - 425s 721ms/step - loss: 0.0032 - acc: 0.9992 - val_loss: 0.0104 - val_acc: 0.9937
Epoch 5/5
120/120 [==============================] - 430s 733ms/step - loss: 0.0032 - acc: 0.9997 - val_loss: 0.0059 - val_acc: 0.9979


In [26]:
for i in range(20):
  test = norm(resize_image(load(X_test[i])))

  print('\n', dir[model(tf.reshape(test, [1, 64, 64, 3]), training=False).numpy().argmax()], dir[y_test[i].argmax()], '\n')


 end end 


 main main 


 main main 


 black black 


 main main 


 battle battle 


 main main 


 end end 


 end end 


 battle battle 


 battle battle 


 black black 


 battle battle 


 battle battle 


 battle battle 


 battle battle 


 battle battle 


 end end 


 team team 


 end end 



In [27]:
test = norm(resize_image(load('/kaggle/input/testing/Screenshot_2023.10.17_18.26.13.351.png')))

dir[model(tf.reshape(test, [1, 64, 64, 3]), training=False).numpy().argmax()]

'team'

In [28]:
model.save('beta_resnet.h5')

In [35]:
new_model = tf.keras.models.load_model('beta_resnet.h5', custom_objects={'Residual':Residual})

In [29]:
import os
os.chdir(r'/kaggle/working')

from IPython.display import FileLink
FileLink(r'beta_resnet.h5')

/kaggle/working/beta_resnet.h5